## GBDT算法实例介绍

4.1.数据介绍

    根据如下数据，预测最后一个样本的身高。
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-1.png)

4.2.模型训练

    4.1.设置参数：
        学习率：learning_rate+0.1
        迭代次数：n_trees=5
        树的深度：max_depth=3

    4.2.开始训练
        （1）初始化弱学习器
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-2.png)

        损失函数为平方损失，因为平方损失函数是一个凸函数，直接求导，导数等于零，得到c
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-3.png)

        令导数为0
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-4.png)

        所以初始化时，c取值为所有训练样本标签值的均值，c=（1.1+1.3+1.7+1.8）/1.475
        此时学习器f0(x)=c=1.475

    2.对迭代论述m=1，2，……，M：
    由于我们设置了迭代次数：n_trees=5，这里的M=5
    计算负梯度，根据上文损失函数为平方损失时，负梯度就是残差，再直白一点就是y与上一轮
    得到的学习器fm-1的差值：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-5.png)

    残差在下表列出：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-6.png)

    此时将残差作为样本的真实值来训练弱学习器f1(x)，即如下表数据：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-7.png)

    接着，寻找回归树的最佳划分节点，遍历每个特征的每个可能取值
    从年龄特征的5开始，到体重特征的70结束，分别计算分类后两组数据的平方损失（Square Error）
    SEl左节点平方损失，SEr右节点平方损失，找到使平方损失和SEsum = SEl + SEr最小
    的那个划分节点，即为最佳划分节点。
    例如：以年龄2为划分节点，将小于21的样本划分到左节点，大于等于21的样本划分为右节点。
    左节点包括x0，x1，右节点包括x2，x3
    SEl = 0.02，SEr=0.005，SEsum=0.025
    SEl = [-0.375-(-0.275)]^2 + [-0.175-(-0.275)]^2 = 0.02
    SEr = [0.225-0.275]^2 + [0.325-0.275]^2 = 0.005
    所有可能划分情况如表所示：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-8.png)

    以上划分点是总平方损失最小为0.025有两个划分点：年龄21和体重60，所以随机选一个作
    为划分点，这里我们选年龄2，现在我们第一棵树长这样：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-9.png)

    我们设置参数中树的深度max_depth=3，现在树的深度只有2，需要再进行一次划分，这次
    划分要对左右两个节点分别进行划分：

    对于左节点，只含有0，1两个样本，根据下表我们选择年龄7划分
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-10.png)

    对于右节点，只含有2，3两个样本，根据下表我们选择年龄30划分（也可以选择体重70）
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-11.png)

    现在，我们第一棵树张这个样子：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-12.png)

    此时，我们的树深度满足了设置，还需要做一件事，给这每个叶子节点分别赋一个参数γ，来拟合残差
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-13.png)

    这里其实和上面初始化学习器是一个道理，平方损失，求导，令导数等于零，化简之后得到每
    个叶子节点的参数γ，其实解释标签值的均值。这个地方的标签值不是原始的γ，而是本轮要拟
    合的标残差y - f0(x)
    根据上述划分结果，为了方便表示，规定从左到右为第1，2，3，4个叶子节点
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-14.png)

    为什么要使用学习率呢？这时Shrinkage的思想，如果而每次都会全部加上（学习率为1）很
    容易一步学到位导致过拟合。
    重复此步骤，知道m>5，最后生成5棵树。
    结果中，0.9杯这个现象，和其学习率有关，这是因为数据简单每棵树都一样，导致每棵树的
    拟合效果都一样，而每棵树都只学习上一颗树残差的0.1倍，导致这棵树只能拟合剩余0.9了。
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-15.png)

    3.得到最后的强学习器：
![jupyter](../../Sources/Pictures/Ensemble_Learning_Algorithms/GBDT算法实例-16.png)

    4.预测样本：
        f0(x) = 1.475
        在f1(x)中，样本4的年龄为25，大于划分节点21岁，又小于30岁，所以被预测为0.2250；
        在f2(x)中，样本4……此处省略……所以被预测为0.2025；
        在f3(x)中，样本4……此处省略……所以被预测为0.1823；
        在f4(x)中，样本4……此处省略……所以被预测为0.1640；
        在f5(x)中，样本4……此处省略……所以被预测为0.1476
    最终预测结果：
        f(x) = 1.475 + 0.1*(0.225+0.205+0.1823+0.164+0.1476) = 1.56714